<a href="https://colab.research.google.com/github/hrumst/ML/blob/master/module_7_competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Строим deep lab v3+ !
https://github.com/bonlime/keras-deeplab-v3-plus

In [0]:
%config IPComplater.greedy = True

In [0]:
!wget http://images.cocodataset.org/zips/train2017.zip  

In [0]:
mkdir train2017 && unzip "./train2017.zip" -d train2017

In [0]:
from google.colab import drive

DRIVE_MOUNT = './DRIVE/'
DRIVE_MOUNT_SAVE_MODEL_PATH = 'My Drive/Colab Notebooks/competition_7/brand-logo-detection-dataset.zip'
drive.mount(DRIVE_MOUNT, force_remount=True)

In [0]:
!cp -r ./train2017 ./DRIVE/My\ Drive/competition_7/coco

In [0]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
import cv2
from tqdm import tqdm_notebook

print('Пример картинки')
plt.figure(figsize=(15,10))
img = cv2.imread("./DRIVE/My Drive/competition_7/coco/train2017/train2017/000000281563.jpg")
label = cv2.imread("./DRIVE/My Drive/competition_7/coco/train2017/train2017/000000281563.png")
# changing to the BGR format of OpenCV to RGB format for matplotlib
plt.subplot(1,3, 1)
plt.imshow(img[:,:,::-1])
plt.title("Image")
plt.subplot(1,3, 2)
plt.imshow(label)
plt.title("Label")
dst = cv2.addWeighted(img,0.3,label,0.8,0)
plt.subplot(1,3, 3)
plt.imshow(dst[:,:,::-1])
plt.title("Blending")
plt.show()

In [0]:
import sys
sys.path.insert(0, './DRIVE/My Drive/competition_7/deeplab_v3_plus')

In [0]:

from model import Deeplabv3
deeplab_model = Deeplabv3(input_shape=(384, 384, 3), classes=4) 

In [0]:
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt

from model import Deeplabv3

# Generates labels using most basic setup.  Supports various image sizes.  Returns image labels in same format
# as original image.  Normalization matches MobileNetV2

trained_image_width=512 
mean_subtraction_value=127.5
image = np.array(Image.open('imgs/image1.jpg'))

# resize to max dimension of images from training dataset
w, h, _ = image.shape
ratio = float(trained_image_width) / np.max([w, h])
resized_image = np.array(Image.fromarray(image.astype('uint8')).resize((int(ratio * h), int(ratio * w))))

# apply normalization for trained dataset images
resized_image = (resized_image / mean_subtraction_value) - 1.

# pad array to square image to match training images
pad_x = int(trained_image_width - resized_image.shape[0])
pad_y = int(trained_image_width - resized_image.shape[1])
resized_image = np.pad(resized_image, ((0, pad_x), (0, pad_y), (0, 0)), mode='constant')

# make prediction
deeplab_model = Deeplabv3()
res = deeplab_model.predict(np.expand_dims(resized_image, 0))
labels = np.argmax(res.squeeze(), -1)

# remove padding and resize back to original image
if pad_x > 0:
    labels = labels[:-pad_x]
if pad_y > 0:
    labels = labels[:, :-pad_y]
labels = np.array(Image.fromarray(labels.astype('uint8')).resize((h, w)))

plt.imshow(labels)
plt.waitforbuttonpress()